# Save `xarray.Datasets` as `pkl` 

In [20]:
import xarray as xr
import s3fs
import _pickle as pickle

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()
from pathlib import Path 
env_path = Path('.') / '.env'

## Utils
**read_dataset**

In [3]:
def read_dataset(dataset_type, group, access_key_id, secret_accsess_key):
    # AWS S3 path
    s3_path = f's3://soils-revealed/{dataset_type}.zarr'
    # Initilize the S3 file system
    s3 = s3fs.S3FileSystem(key=access_key_id, secret=secret_accsess_key)
    store = s3fs.S3Map(root=s3_path, s3=s3, check=False)
    # Read Zarr file
    ds = xr.open_zarr(store=store, group=group, consolidated=True)
    
    # Change coordinates names
    ds = ds.rename({'x': 'lon', 'y': 'lat'})
    
    # Change depth coord from 0 to 1 dimensional array
    depths = ds.coords.get('depth').values
    if depths.ndim == 0: 
        ds = ds.squeeze().drop("depth")
        ds = ds.assign_coords({"depth": np.array([depths])})
        
    return ds 

**save_object**

In [9]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, -1)

## Save `xarray.Datasets`

In [21]:
datasets = {'experimental-dataset': ['stocks', 'concentration']}

for dataset_type, groups in datasets.items():
    for group in groups:
        ds = read_dataset(dataset_type = dataset_type, group = group, 
                          access_key_id = os.getenv("S3_ACCESS_KEY_ID"), 
                          secret_accsess_key = os.getenv("S3_SECRET_ACCESS_KEY"))    
        
        save_object(ds, f'../data/{dataset_type}_{group}.pkl')